In [686]:
import PyPDF2
import nltk
import re
import numpy as np
from PyPDF2 import PdfFileWriter, PdfFileReader
from nltk import sent_tokenize,word_tokenize
from nltk.tokenize  import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.probability import FreqDist
from string import punctuation
from nltk.corpus import stopwords
from nltk.text import Text
from string import punctuation
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [687]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options 
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import sys
import time
import shutil
import re
import itertools


chrome_options = Options()
preferences = {"directory_upgrade": True,"safebrowsing.enabled": True }
chrome_options.add_experimental_option("prefs", preferences)

chrome_options.add_extension(r"D:\Threat_intel_services\APT_collections\nlp_crawling\extension_2_7_14_0.crx")
driver = webdriver.Chrome(r"D:\RHYMcygiene\database\chromedriver.exe",options=chrome_options)

In [384]:
links=['https://www.proofpoint.com/us/threat-insight/post/unraveling-ThreadKit-new-document-exploit-builder-distribute-The-Trick-Formbook-Loki-Bot-malware',
      'https://www.proofpoint.com/us/threat-insight/post/new-modular-downloaders-fingerprint-systems-part-3-cobint',
      'https://malpedia.caad.fkie.fraunhofer.de/actor/cobalt',
      'https://www.fireeye.com/blog/threat-research/2013/08/survival-of-the-fittest-new-york-times-attackers-evolve-quickly.html',
      'https://www.sophos.com/en-us/medialibrary/PDFs/technical-papers/sophoslabs-uncut-2020-threat-report.pdf']

In [385]:
site='https://www.printfriendly.com/'

In [361]:
# function to extract text from all pages
def raw_pdf(x):
    page=[]
    for i in  range(0,x.numPages): 
        pages=x.getPage(i) 
        story=pages.extractText() 
        k= ' '.join (story.split())
        page.append(k)
    return page

In [362]:
def download_file(url, filename):
    if not os.path.isfile(filename):
        print('Downloading File')
        response = requests.get(url)
        if response.status_code == 200:
            with open(filename, 'wb') as file:
                for chunk in response:
                    file.write(chunk)
    else:
            print('file exists')

In [697]:
def CVE_ID(text):

    pattern= 'CVE-\d{4}-\d{4,7}'
    result = re.findall(pattern, text ,re.I)
    result=set(result)
    return (result)

In [648]:
def email_finder(text):
    a=re.compile(r'[\w\.-]+@[\w\.-]+')
    result=a.findall(text)
    return result

In [684]:
def find_ip_address(text):
    pat = re.compile("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")
    result=pat.findall(text)
    
    return result

In [664]:
def extract_hashes(text):
    regex_list = {

    'wordpress_md5': '\$P\$[\w\d./]+',
    'phpBB3_md5': '\$H\$[\w\d./]+',
    'sha1':  '(?<!\w)[a-f\d]{40}(?!\w)',
    'md5':  '(?<!\w)[a-f\d]{32}(?!\w)',
    'sha256':  '(?<!\w)[a-f\d]{64}(?!\w)',
    'sha512':  '(?<!\w)[a-f\d]{128}(?!\w)',
    'mysql':  '(?<!\w)[a-f\d]{16}(?!\w)',
    'mysql5': '\*[A-F\d]{40}'

    }

    result = {}

    for format in regex_list.keys():
        hashes = []
        regex = re.compile(regex_list[format])
        hashes = regex.findall(text)
        if hashes:
            result[format] = hashes

    return result

In [386]:
cve=[]
hashes=[]
files=[]
for link in links:
    if link.endswith('.pdf'):
        download_file(link,filename=os.getcwd()+os.sep+'reports_pdf'+os.sep+link.split('/')[-1])
        test=PdfFileReader(open(os.getcwd()+os.sep+'reports_pdf'+os.sep+link.split('/')[-1],'rb'))
        text=raw_pdf(test)
        text=' '.join(text)
    else:
        driver.get(site)
        text_box=driver.find_element_by_xpath("/html/body/div[1]/div[2]/div/div[1]/div[1]/form/input[2]")
        text_box.send_keys(link)
        driver.find_element_by_xpath("/html/body/div[1]/div[2]/div/div[1]/div[1]/form/input[3]").click()
        time.sleep(3)
        source_code=driver.page_source
        soup = BeautifulSoup(source_code, 'html.parser')
        text=soup.find('div',id='pf-body').text
        
    cve.append(CVE_ID(text))
    hashes.append(extract_hashes(text))

In [685]:
list(set(itertools.chain.from_iterable(cve)))

TypeError: 'list' object is not callable

In [591]:
extensions='exe|pif|application|gadget|msi|msp|com|scr|hta|cpl|msc|jar|bat|cmd|vb|vbe|js|jse|ws|wsc|ps1|msh|scf|lnk|inf|reg|doc|docm'

In [595]:
link='https://blog.malwarebytes.com/threat-analysis/2019/10/magecart-group-4-a-link-with-cobalt-group/'

In [597]:
driver.get(site)
text_box=driver.find_element_by_xpath("/html/body/div[1]/div[2]/div/div[1]/div[1]/form/input[2]")
text_box.send_keys(link)
driver.find_element_by_xpath("/html/body/div[1]/div[2]/div/div[1]/div[1]/form/input[3]").click()
time.sleep(3)
source_code=driver.page_source
soup = BeautifulSoup(source_code, 'html.parser')
text=soup.find('div',id='pf-body').text

In [762]:
text=driver.page_source
soup = BeautifulSoup(text, 'html.parser')
text=soup.find('div',id='pf-body').text

In [763]:
extract_hashes(text)

{'sha256': ['444c63bb794abe3d2b524e0cb2c8dcc174279b23b1bce949a7125df9fab25c1c',
  '1c1a6bb0937c454eb397495eea034e00d1f7cf4e77481a04439afbc5b3503396',
  '988d430ce0e9f19634cf7955eac6eb03e3b7774b788010c2a9742b38016d1ebf',
  '1d0aae6cff1f7a772fac67b74a39904b8b9da46484b4ae8b621a6566f7761d16',
  '57f65ecb239833e5a4b2441e3a2daf3513356d45e1d5c311baeb31f4d503703e',
  '852f11e5131d3dab9812fd8ce3cd94c1333904f38713ff959f980a168ef0d4ce',
  '19dc9b93870ddc3beb7fdeea2980c95edc489040e39381d89d0dfe0a825a1570',
  '020ba5a273c0992d62faa05144aed7f174af64c836bf82009ada46f1ce3b6eee',
  '07a3355f81ff69a197c792847d0783bfc336181d66d3a36e6b548d0dbd9f5a9a',
  '161ba501b4ea6f7c2c8d224e55e566fef95064e1ed059d8287bc07e790f740e8',
  '19dc9b93870ddc3beb7fdeea2980c95edc489040e39381d89d0dfe0a825a1570',
  'dc8425f8c966708b1a3c26f0545664ccbf853852af401b91ae7f29d351e2649c',
  'dc8425f8c966708b1a3c26f0545664ccbf853852af401b91ae7f29d351e2649c',
  '45310fcc9f9ef367f16bed4c4ba4c51d7eb72550082cd572f6a5636227514d70',
  'df18e99

In [691]:
def email_finder(text):
    a=re.compile(r'[\w\.-]+@[\w\.-]+')
    result=a.findall(text)
    return result

In [764]:
def CVE_ID(text):

    pattern= 'CVE-\d{4}-\d{4,7}'
    result = re.findall(pattern, text ,re.I)
    result=set(result)
    return result

In [765]:
a=CVE_ID(text)

In [766]:
a

set()

In [749]:
text='CVE-2017–11882'
pattern=  re.compile('CVE-\d{4}-\d{4,7}')
pattern.findall(text)

[]

In [748]:
result

[]

In [745]:
text

'including CVE-2017–11882, CVE-2018–0802, and CVE-2018–8174'

In [693]:
def find_ip_address(text):
    pat = re.compile("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")
    result=pat.findall(text)
    
    return result

In [694]:
find_ip_address(text)

[]

In [349]:
download_file('https://www.cse.msu.edu/~bajpaipr/resources/apt1_one_of_chinas_cyber_espionage_units_Pranshu_Bajpai.pdf',filename=os.getcwd()+os.sep+'reports_pdf'+os.sep+'sample.pdf')

In [354]:
# function to extract text from all pages
def raw_pdf(x):
    page=[]
    for i in  range(0,x.numPages): 
        pages=x.getPage(i) 
        story=pages.extractText() 
        k= ' '.join (story.split())
        page.append(k)
    return page

In [353]:
# importing of regulation pdf
test=PdfFileReader(open(r'reports_pdf/sample.pdf','rb'))

In [355]:
# extraxting data
data=raw_pdf(test)


In [323]:
source_code=driver.page_source
soup = BeautifulSoup(source_code, 'html.parser')
text=soup.find('div',id='pf-body').text

{}

In [ ]:
df=pd.read_html(driver.current_url)

In [110]:
df[0]

,IOC,IOC Type,Description
0,8e1c6f44b02e72b1c1c9af0ffdcee0fbe67fb8ee370bc6...,SHA256,ThreadKit document from June 2017 example
1,5528b7baa2be1b02140378b076efdbb413ea5e7b05541d...,SHA256,ThreadKit HTA from June 2017 example
2,030ce92b54ae176b116b6d1baa4769d0298f7283dcc163...,SHA256,ThreadKit VBS from June 2017 example
3,2bb9d0d8166a8d330cb3c5be6fb60539fe29e05cc3acb4...,SHA256,ThreadKit document from October 2017 example
4,094ec9e51a559f93e920643a1167de6ee966076af6ab89...,SHA256,ThreadKit Soap file from October 2017 example
5,877da1bc247c3a0406e29f1cbe50e89b6b53f070957b2d...,SHA256,ThreadKit HTA file from October 2017 example
6,c73c570208059e0a7b712d835c0de1def72d346a0d6235...,SHA256,ThreadKit document from November 2017 example
7,53e8890f0d002d9611675419b3d8d0899b599c59f4557e...,SHA256,ThreadKit document from February 2018 example
